# Hosting Strands + LangGraph agent with Amazon Bedrock models in Amazon Bedrock AgentCore Runtime

## Overview

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. 

We will focus on a LangGraph with Amazon Bedrock model example. For Strands Agents with Amazon Bedrock model check [here](../01-strands-with-bedrock-model)
and for a Strands Agents with an OpenAI model check [here](../03-strands-with-openai-model).

### Tutorial Details

| Information         | Details                                                                      |
|:--------------------|:-----------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                               |
| Agent type          | Single                                                                       |
| Agentic Framework   | Strands + LangGraph                                                                    |
| LLM model           | Anthropic Claude Sonnet 3                                                    |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands + LangGraph and Amazon Bedrock Model |
| Tutorial vertical   | Cross-vertical                                                               |
| Example complexity  | Easy                                                                         |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                 |

### Tutorial Architecture

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime. 

For demonstration purposes, we will  use a LangGraph agent using Amazon Bedrock models

In our example we will use a very simple agent with two tools: `get_weather` and `get_time`. 

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Amazon Bedrock models
* Using LangGraph


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* LangGraph
* Docker running

## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

The architecture here will look as following:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="60%"/>
</div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../../../.."
sys.path.append(os.path.abspath(module_path))

## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### Strands Agent SDK + LangGraph with Amazon Bedrock model
Let's start with our Strands Agent SDK + LangGraph using Amazon Bedrock model. Other examples with different frameworks and models are available in the parent directories

### Dockerfile


In [ ]:
%%writefile runtime.py

"""
AgentCore Runtime for Bedrock-Manus Multi-Agent System
Converted from main.py to use AgentCore Runtime API with unified event streaming
"""
import os
import shutil
import asyncio
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from src.graph.builder import build_graph
from src.utils.strands_sdk_utils import strands_utils

# Import event queue for unified event processing
from src.utils.event_queue import clear_queue

app = BedrockAgentCoreApp()

def remove_artifact_folder(folder_path="./artifacts/"):
    """
    ./artifact/ 폴더가 존재하면 삭제하는 함수
    
    Args:
        folder_path (str): 삭제할 폴더 경로
    """
    if os.path.exists(folder_path):
        print(f"'{folder_path}' 폴더를 삭제합니다...")
        try:
            shutil.rmtree(folder_path)
            print(f"'{folder_path}' 폴더가 성공적으로 삭제되었습니다.")
        except Exception as e: 
            print(f"오류 발생: {e}")
    else:
        print(f"'{folder_path}' 폴더가 존재하지 않습니다.")

def _setup_execution():
    """Initialize execution environment"""
    remove_artifact_folder()
    clear_queue()
    print("\n=== Starting AgentCore Runtime Event Stream ===")

def _print_conversation_history():
    """Print final conversation history"""
    print("\n=== Conversation History ===")
    from src.graph.nodes import _global_node_states
    shared_state = _global_node_states.get('shared', {})
    history = shared_state.get('history', [])
    
    if history:
        for hist_item in history:
            print(f"[{hist_item['agent']}] {hist_item['message']}")
    else:
        print("No conversation history found")

@app.entrypoint
async def graph_streaming_execution(payload):
    """
    Execute full graph streaming workflow through AgentCore Runtime
    Queue-only event processing compatible with AgentCore API
    """
    # Get user query from payload
    user_query = payload.get("prompt", "")
    
    if not user_query:
        # Use default query if none provided
        user_query = "너가 작성할 것은 moon market 의 판매 현황 보고서야. 세일즈 및 마케팅 관점으로 분석을 해주고, 차트 생성 및 인사이트도 뽑아서 pdf 파일로 만들어줘. 분석대상은 './data/Dat-fresh-food-claude.csv' 파일 입니다. Coder 에이전트가 할일은 최대한 작게 해줘. 왜냐하면 reporter 에이전트 테스트 중이라 빨리 코더 단계를 넘어 가야 하거든. 부탁해."
    
    _setup_execution()
    
    # Build graph and use stream_async method
    graph = build_graph()
    event_count = 0
    
    # Stream events from graph execution
    async for event in graph.stream_async({
        "request": user_query,
        "request_prompt": f"Here is a user request: <user_request>{user_query}</user_request>"
    }):
        event_count += 1
        # Add AgentCore runtime metadata
        event["event_id"] = event_count
        event["runtime_source"] = "bedrock_manus_agentcore"
        
        # Mark final event
        if event.get("type") == "workflow_complete":
            event["total_events"] = event_count
            event["message"] = "All events processed through global queue via AgentCore Runtime"
        
        yield event
    
    _print_conversation_history()
    print("=== AgentCore Runtime Event Stream Complete ===")


if __name__ == "__main__":
    app.run()
    
    


## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Creating runtime role

Before starting, let's create an IAM role for our AgentCore Runtime. We will do so using the utils function pre-developed for you.

In [4]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.join(utils_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from src.utils.agentcore import create_agentcore_role

agent_name="runtime"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

sys.path[0]: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications
Role already exists -- deleting and creating it again
policies: {'PolicyNames': ['AgentCorePolicy'], 'IsTruncated': False, 'ResponseMetadata': {'RequestId': 'c0789510-86f0-434a-8830-3b9522bd314f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 03 Sep 2025 09:20:05 GMT', 'x-amzn-requestid': 'c0789510-86f0-434a-8830-3b9522bd314f', 'content-type': 'text/xml', 'content-length': '380'}, 'RetryAttempts': 0}}
deleting agentcore-runtime-role
recreating agentcore-runtime-role
attaching role policy agentcore-runtime-role


### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

**Signature:**
```python
agentcore_runtime.configure(
    entrypoint: str,
    execution_role: Optional[str] = None,
    agent_name: Optional[str] = None,
    requirements: Optional[List[str]] = None,
    requirements_file: Optional[str] = None,
    ecr_repository: Optional[str] = None,
    container_runtime: Optional[str] = None,
    auto_create_ecr: bool = True,
    auto_create_execution_role: bool = False,
    authorizer_configuration: Optional[Dict[str, Any]] = None,
    region: Optional[str] = None,
    protocol: Optional[Literal['HTTP', 'MCP']] = None,
) -> bedrock_agentcore_starter_toolkit.operations.runtime.models.ConfigureResult
```

**Docstring:**  
Configure Bedrock AgentCore from notebook using an entrypoint file.

**Args:**
- **entrypoint**: Path to Python file with optional Bedrock AgentCore name (e.g., "handler.py" or "handler.py:bedrock_agentcore")
- **execution_role**: AWS IAM execution role ARN or name (optional if auto_create_execution_role=True)
- **agent_name**: name of the agent
- **requirements**: Optional list of requirements to generate requirements.txt
- **requirements_file**: Optional path to existing requirements file
- **ecr_repository**: Optional ECR repository URI
- **container_runtime**: Optional container runtime (docker/podman)
- **auto_create_ecr**: Whether to auto-create ECR repository
- **auto_create_execution_role**: Whether to auto-create execution role (makes execution_role optional)
- **authorizer_configuration**: JWT authorizer configuration dictionary
- **region**: AWS region for deployment
- **protocol**: agent server protocol, must be either HTTP or MCP

In [5]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    agent_name=agent_name,
    entrypoint="runtime.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region
)
response

Entrypoint parsed: file=/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/runtime.py, bedrock_agentcore_name=runtime
Configuring BedrockAgentCore agent: runtime
Generated Dockerfile: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/Dockerfile
Generated .dockerignore: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/.dockerignore
Keeping 'runtime' as default agent
Bedrock AgentCore configured: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/Dockerfile'), dockerignore_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/.dockerignore'), runtime='Docker', region='us-west-2', account_id='615299776985', execution_role='arn:aws:iam::615299776985:role/agentcore-runtime-role', ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [6]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'runtime' to account 615299776985 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: runtime
✅ ECR repository available: 615299776985.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-runtime
Using execution role from config: arn:aws:iam::615299776985:role/agentcore-runtime-role
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: runtime
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-d92c6a81b2


✅ Reusing existing ECR repository: 615299776985.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-runtime


Reusing existing CodeBuild execution role: arn:aws:iam::615299776985:role/AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-d92c6a81b2
Using .dockerignore with 43 patterns
Uploaded source to S3: runtime/source.zip
Updated CodeBuild project: bedrock-agentcore-runtime-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.0s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 8.2s
🔄 DOWNLOAD_SOURCE started (total: 9s)
✅ DOWNLOAD_SOURCE completed in 1.0s
🔄 INSTALL started (total: 10s)
✅ INSTALL completed in 1.0s
🔄 BUILD started (total: 11s)
✅ BUILD completed in 37.0s
🔄 POST_BUILD started (total: 48s)
✅ POST_BUILD completed in 45.2s
🔄 FINALIZING started (total: 94s)
✅ FINALIZING completed in 1.0s
🔄 COMPLETED started (total: 95s)
✅ COMPLETED completed in 0.0s
🎉 CodeBuild completed successfully in 1m 34s
CodeBuild completed successfully
✅ CodeBuild project configuration saved
Deploying to

In [7]:
import platform
print(f"Machine: {platform.machine()}")
print(f"Processor: {platform.processor()}")
print(f"Architecture: {platform.architecture()}")
print(f"Platform: {platform.platform()}")

Machine: aarch64
Processor: aarch64
Architecture: ('64bit', 'ELF')
Platform: Linux-6.8.0-1033-aws-aarch64-with-glibc2.35


### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [8]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: runtime


'READY'

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [9]:
import json
import boto3
from boto3.session import Session
from src.utils.strands_sdk_utils import strands_utils

boto_session = Session()
region = boto_session.region_name

def parse_sse_data(sse_bytes):
    if not sse_bytes or len(sse_bytes) == 0:
        return None

    try:
        text = sse_bytes.decode('utf-8').strip()
        if not text or text == '': return None

        if text.startswith('data: '):
            json_text = text[6:].strip()
            if json_text: return json.loads(json_text)
        else:
            return json.loads(text)

    except Exception as e:
        # 디버깅용으로만 출력
        # print(f"Parse error: {e}")
        pass

    return None
      

agent_arn = launch_result.agent_arn
from botocore.config import Config

my_config = Config(
    connect_timeout=60*100,
    read_timeout=60*5,
)

agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region,
    config=my_config,
)

#agent_arn = "arn:aws:bedrock-agentcore:us-west-2:615299776985:runtime/runtime-GxV7G995xb"

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    #payload=json.dumps({"prompt": "What is the weather now?"})
    payload=json.dumps({"prompt": '이것은 아마존 상품판매 데이터를 분석하고 싶습니다. 분석대상은 "./data/Dat-fresh-food-claude.csv" 파일 입니다. 데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요. 분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요. 데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요. 분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요. 최종 리포트는 pdf 형태로 저장해 주세요.'})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for event in boto3_response["response"].iter_lines(chunk_size=1):
        event = parse_sse_data(event)
        if event is None:  # None 체크 추가
            continue
        else:
            strands_utils.process_event_for_display(event)
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            print ("6", event)
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

handoff_to_planner: I'll need to consult our planning system for this request.

This request involves complex data analysis tasks including:
- Reading and analyzing Amazon sales data
- Performing exploratory data analysis
- Analyzing sales trends and variable relationships
- Creating visualizations and charts
- Generating marketing insights
- Producing a PDF report

I'll pass this to our planning system to handle these technical requirements properly.This user request asks for an analysis of Amazon fresh food product data from a CSV file located at "./data/Dat-fresh-food-claude.csv". The goal is to extract marketing insights through a variety of analyses including basic exploratory data analysis (EDA), sales trend analysis, variable relationship analysis, and potentially additional analyses based on initial findings. The request also specifies that the final report should include visualizations/charts and be saved as a PDF.

Let me break down what needs to be done and plan accordingly:

ReadTimeoutError: Read timeout on endpoint URL: "None"

### Processing invocation results

We can now process our invocation results to include it in an application

In [ ]:
from IPython.display import Markdown, display
import json
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [ ]:
import boto3
import json
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    #payload=json.dumps({"prompt": "What is the weather now?"})
    payload=json.dumps({"prompt": "안녕 나는 장동진이라고 해. 만나서 반가워 나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어."})
    #agentcore_runtime.invoke({"prompt": "안녕 나는 장동진이라고 해. 만나서 반가워 나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어."})
)
# if "text/event-stream" in boto3_response.get("contentType", ""):
#     content = []
#     for line in boto3_response["response"].iter_lines(chunk_size=1):
#         if line:
#             line = line.decode("utf-8")
#             if line.startswith("data: "):
#                 line = line[6:]
#                 logger.info(line)
#                 content.append(line)
#     display(Markdown("\n".join(content)))
# else:
#     try:
#         events = []
#         for event in boto3_response.get("response", []):
#             events.append(event)
#     except Exception as e:
#         events = [f"Error reading EventStream: {e}"]
#     display(Markdown(json.loads(events[0].decode("utf-8"))))

In [ ]:
boto3_response

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)
policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_iam_role['Role']['RoleName']
)

# Congratulations!